In [5]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#from fancyimpute import KNN

#preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

#train test split
from sklearn.model_selection import train_test_split

#sklearn models
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA

#Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

#one hot
from sklearn.preprocessing import OneHotEncoder

from collections import Counter

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Code for knn imputation
### Did not work: memory error issue

In [6]:
data = pd.read_csv("../Dataset/fully_encoded.csv", low_memory=False)

# Correlation Plot
### Removed columns with correlation against case status less than 0.0001

In [7]:
correlation = data.corr()
plt.matshow(correlation)

In [8]:
corr_case_status = correlation["case_status"]
low_val = corr_case_status>0.0001
Counter(list(low_val))

Counter({False: 57, True: 54})

In [9]:
columns = list(data.columns)
new_data = pd.DataFrame()
for a in columns:
    if a=='class_of_admission' or correlation['case_status'][a]>0.0001:
        new_data[a] = data[a]

# Imputation

In [10]:
new_data.drop(['class_of_admission', 'case_status'], axis = 1, inplace = True)

## KNN attempted for imputation. Did not work due to memory issue

## Imputation with mean value

In [11]:
new_data = new_data.fillna(value = new_data.mean().to_dict())

In [12]:
X_normalized = StandardScaler().fit_transform(new_data)
pca = PCA(n_components = 40)
principalComponents = pca.fit_transform(X_normalized)
principalDf = pd.DataFrame(data = principalComponents)

print("explained variance: ", pca.explained_variance_ratio_)
print("total: ", sum(pca.explained_variance_ratio_)*100)

explained variance:  [0.03683722 0.03155411 0.02712954 0.02285996 0.02261256 0.02245898
 0.0219014  0.02142511 0.02121099 0.02070546 0.02031272 0.02014971
 0.02000689 0.0199427  0.01969017 0.01966167 0.01952625 0.01943864
 0.01938724 0.01922036 0.01912021 0.01906894 0.01902004 0.0190001
 0.01896258 0.01892937 0.01891178 0.01888543 0.01887545 0.0188138
 0.0187728  0.01873756 0.01869577 0.01865467 0.01855741 0.0184743
 0.01843658 0.01822532 0.01806544 0.01788895]
total:  82.0128170555737


# Results of PCA
<table>
    <tr>
        <td>Number of Components</td><td>Explained Variance %</td>
    </tr>
    <tr>
        <td>25</td><td>53.5489</td>
    </tr>
    <tr>
        <td>30</td><td>63.1431</td>
    </tr>
    <tr>
        <td>35</td><td>72.6216</td>
    </tr>
    <tr>
        <td>40</td><td>82.004</td>
    </tr>
    <tr>
        <td>45</td><td>90.574</td>
    </tr>
    <tr>
        <td>50</td><td>97.7323</td>
    </tr>
</table>

In [20]:
principalDf['case_status'] = data['case_status']
def fun(x):
    d = {'H-1B':0,'L-1':1,'F-1':2}
    return d[x]
principalDf['class_of_admission'] = data['class_of_admission'].apply(fun)

In [22]:
principalDf.to_csv("../Dataset/afterPCA.csv")